In [2]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")

import IPython
import IPython.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from load_data import merged_df

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

2.8.0


In [136]:
#load the dataset, slice a smaller amount for now
REDUCED_DATESET_SIZE = 1_000_000
df = merged_df()
df = df[:REDUCED_DATESET_SIZE]

loading train data...
loading gas_prices...
loading electricity_prices...
loading forecast_weather...
merging train and gas_prices...
merging electricity_prices...
merging forecast_weather...


In [137]:
#test/train split
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

#drop the target column 
train_labels = train_dataset['target']
test_labels = test_dataset['target']

train_features = train_dataset.drop(columns=['target'])
test_features = test_dataset.drop(columns=['target'])

num_features = len(test_features.columns.values)

In [138]:
#write feature_names to .csv file
import csv
feature_names = list(test_features.columns.values)
with open("../models/dnn_feature_names.csv", 'w') as f:
    write = csv.writer(f)
    write.writerow(feature_names)

In [139]:
#create a normalization layer
normalizer = tf.keras.layers.Normalization(input_dim=num_features)

#normalize the train_features by subtracting the mean and dividing by the std deviation
normalizer.adapt(np.array(train_features))

2023-12-16 17:40:59.316514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-12-16 17:40:59.331519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [140]:
#build the three layer model with initial normalization layer
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

#fit the model
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_9 (Normalizat  (None, 23)               47        
 ion)                                                            
                                                                 
 dense_12 (Dense)            (None, 64)                1536      
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,808
Trainable params: 5,761
Non-trainable params: 47
_________________________________________________________________
Epoch 1/2
   12/20000 [..............................] - ETA: 1:34 - loss: 200.7637  

2023-12-16 17:41:56.002856: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19999/20000 [============================>.] - ETA: 0s - loss: 121.1733

2023-12-16 17:43:22.202792: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20000/20000 [==============================] - 99s 5ms/step - loss: 121.1677 - val_loss: 110.6825
Epoch 2/2
20000/20000 [==============================] - 97s 5ms/step - loss: 97.4114 - val_loss: 86.6467


In [141]:
# save the model to disk
import pickle
dnn_model_filename = '../models/dnn_model.sav'
pickle.dump(dnn_model, open(dnn_model_filename, 'wb'))

INFO:tensorflow:Assets written to: ram://3a7ffb65-0a04-4326-a011-edc98089045d/assets


In [142]:
#save model using keras
dnn_model.save('../models/dnn_model.keras')